In [1]:
# Compute Call and Put Probabilities
# Compute the Return on Margin per year
# Compute how far the Strike Price is from Yearly High (for Call) and Low (for Put) in %
# Filter out Last Traded Price > BS
# Filter out only those that have Open Interests
# Reverse Sort the result on RoM * Probability

# Fish out Probabilities > 92%

# Read the file
import pandas as pd
df = pd.read_excel('NSE.xlsx')
df.head()

# Compute Call and Put Probabilities
df['pProb'] = 1-df.cProb

# Compute Return on Margins
df['cRoM'] = df.Call_LTP/(df.MarginRate*df.Underlying)*365/df.DTE  # Call Return on Margin
df['pRoM'] = df.Put_LTP/(df.MarginRate*df.Underlying)*365/df.DTE  # Put Return on Margin

# How close Price is from Hi52 (for calls), and Lo52 (for Puts)
# Negative values for 52Delta is better - as they are beyond Hi/Lo52
df['Delta52'] = (df.Hi52-df.Strike)/df.Strike                                 # for Calls
df.loc[df.Strike < df.Underlying, "Delta52"] = (df.Strike-df.Lo52)/df.Strike  # for Puts

# Filter out LTP > BS and there is some OI on these
df = df[((df.Call_LTP > df.BSCall) | (df.Put_LTP > df.BSPut)) &
        ((df.Call_OI > 0) | (df.Put_OI > 0))]

# Filter out Call_LTP = 0 for Strike Price > Underlying and 
#            Put_LTP = 0 for Strike Price < Underlying
df = df[((df.Call_LTP > 0) & (df.Strike > df.Underlying)) |
   ((df.Put_LTP > 0) & (df.Strike < df.Underlying))]

# Calculate the result on RoM * Probability and reverse sort on it
df['RoMPoP'] = df.cProb * df.cRoM                                 # For Palls
df.loc[df.Strike < df.Underlying, "RoMPoP"] = df.pProb * df.pRoM  # For Puts
df = df.sort_values(by=['RoMPoP'], ascending=False).reset_index(drop=True)

# Identify the Calls and Puts
c = df[df.cProb>0.98]
p = df[df.pProb>0.95]

# Get only the negative Delat52s
calls = c[c.Delta52<0]
puts = p[p.Delta52<0]
call=calls[['Symbol', 'Expiry', 'Strike', 'Call_LTP', 'Call_BidQty', 'Underlying', 'cRoM', 'cProb', 'Call_OI', 'Hi52', 'Delta52', 'RoMPoP', 'Call_AskPrice', 'Lot']]
put=puts[['Symbol', 'Expiry', 'Strike', 'Put_LTP','Put_BidQty', 'Underlying', 'pRoM', 'pProb', 'Put_OI', 'Lo52', 'Delta52', 'RoMPoP', 'Put_AskPrice', 'Lot']]

In [2]:
calls[['Symbol', 'Expiry', 'Strike', 'Call_LTP', 'Underlying', 'Hi52', 'Delta52', 'RoMPoP', 'cRoM', 'cProb', 'Call_OI', 'Call_AskPrice', 'Lot']]
puts[['Symbol', 'Expiry', 'Strike', 'Expiry', 'Put_LTP', 'Underlying', 'Lo52', 'Delta52', 'RoMPoP', 'pRoM', 'pProb', 'Put_OI', 'Put_AskPrice', 'Lot']]

,Symbol,Expiry,Strike,Expiry,Put_LTP,Underlying,Lo52,Delta52,RoMPoP,pRoM,pProb,Put_OI,Put_AskPrice,Lot
6,SOUTHBANK,26APR2018,17.5,26APR2018,2.30,24.80,19.75,-0.128571,5.772417,5.821892,0.991502,33141,1.15,33141
7,PTC,28MAR2018,70.0,28MAR2018,3.00,90.35,84.00,-0.200000,5.649346,5.662540,0.997670,360000,0.40,8000
19,MANAPPURAM,28MAR2018,77.5,28MAR2018,2.75,102.35,82.10,-0.059355,4.080858,4.097190,0.996014,360000,0.65,6000
40,INDIACEM,28MAR2018,110.0,28MAR2018,3.05,145.15,141.50,-0.286364,3.294036,3.305172,0.996631,3500,2.65,3500
46,DCBBANK,28MAR2018,115.0,28MAR2018,3.00,160.00,151.70,-0.319130,3.220541,3.220588,0.999985,9000,0.80,4500
81,NBCC,28MAR2018,150.0,28MAR2018,3.00,182.25,160.55,-0.070333,2.799989,2.827403,0.990304,180000,2.75,3000
91,NBCC,28MAR2018,140.0,28MAR2018,2.85,182.25,160.55,-0.146786,2.683970,2.686032,0.999232,165000,2.70,3000
106,MANAPPURAM,28MAR2018,80.0,28MAR2018,1.75,102.35,82.10,-0.026250,2.582703,2.607303,0.990565,150000,1.20,6000
118,EXIDEIND,28MAR2018,145.0,28MAR2018,3.05,210.20,192.40,-0.326897,2.492304,2.492304,1.000000,4000,1.00,4000
134,DCBBANK,28MAR2018,140.0,28MAR2018,2.35,160.00,151.70,-0.083571,2.408438,2.522794,0.954671,184500,0.90,4500


In [2]:
writer = pd.ExcelWriter('puts_calls.xlsx')
put.to_excel(writer, 'Puts', index=False , freeze_panes=(1,1))
call.to_excel(writer, 'Calls', index=False, freeze_panes=(1,1))
writer.save()